In [5]:
pip install -r requirements.txt

  Using cached langchain-0.0.333-py3-none-any.whl.metadata (16 kB)
  Using cached langsmith-0.0.63-py3-none-any.whl.metadata (10 kB)
  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
  Using cached tiktoken-0.5.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (6.6 kB)
Using cached langchain-0.0.333-py3-none-any.whl (2.0 MB)
Using cached langsmith-0.0.63-py3-none-any.whl (45 kB)
Using cached openai-0.28.1-py3-none-any.whl (76 kB)
Using cached tiktoken-0.5.2-cp311-cp311-macosx_10_9_x86_64.whl (1.0 MB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.7.0
    Uninstalling tiktoken-0.7.0:
      Successfully uninstalled tiktoken-0.7.0
  Attempting uninstall: openai
    Found existing installation: openai 1.30.1
    Uninstalling openai-1.30.1:
      Successfully uninstalled openai-1.30.1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.59
    Uninstalling langsmith-0.1.59:
      Successfully uninstalled langsmith-0.1.59
  A

In [6]:
pip install langchain_community

  Using cached langchain-0.2.0-py3-none-any.whl.metadata (13 kB)
  Using cached langsmith-0.1.59-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.2.0-py3-none-any.whl (973 kB)
Using cached langsmith-0.1.59-py3-none-any.whl (121 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.63
    Uninstalling langsmith-0.0.63:
      Successfully uninstalled langsmith-0.0.63
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.333
    Uninstalling langchain-0.0.333:
      Successfully uninstalled langchain-0.0.333
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install langchain_openai

  Using cached openai-1.30.1-py3-none-any.whl.metadata (21 kB)
  Using cached tiktoken-0.7.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (6.6 kB)
Using cached openai-1.30.1-py3-none-any.whl (320 kB)
Using cached tiktoken-0.7.0-cp311-cp311-macosx_10_9_x86_64.whl (961 kB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.2
    Uninstalling tiktoken-0.5.2:
      Successfully uninstalled tiktoken-0.5.2
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1
Note: you may need to restart the kernel to use updated packages.


In [8]:
import dotenv

dotenv.load_dotenv()

True

In [9]:
pip install tree-sitter tree-sitter-languages

Note: you may need to restart the kernel to use updated packages.


In [11]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [13]:
# Clone
# repo_path = "/Users/swaramenon/Documents/nlq1-app1"
# repo = Repo.clone_from("https://github.com/redapt/pyspark-s3-parquet-example", to_path=repo_path)
repo_path = "/Users/swaramenon/Documents/graphql"
repo = Repo.clone_from("https://github.com/sogko/golang-relay-starter-kit", to_path=repo_path)

In [34]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path + "/",
    glob="**/*",
    suffixes=[".go",".json",".js"],
    # exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(parser_threshold=500),
)
documents = loader.load()
len(documents)

11

In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.GO, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

31

In [36]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [37]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

qa = create_retrieval_chain(retriever_chain, document_chain)

In [38]:
question = "explain the repo"
result = qa.invoke({"input": question})
result["answer"]

'The repository named "golang-relay-starter-kit" is a private repository designed to help developers quickly start using Relay with Golang. It is hosted on GitHub under the user sogko. The current version is 0.1.0.\n\nThe repository includes scripts to start the program and update the schema. It also lists dependencies which include various versions of babel, classnames, express, react, webpack, and more.\n\nThe code snippets provided show a basic example of how to use Relay with React and Golang. It includes a simple React component that displays a list of widgets, and a server setup using express, webpack, and GraphQL.\n\nThe GraphQL schema is also defined, including types for User and Widget, and it demonstrates how to fetch an object by its ID and a person\'s collection of widgets. It also provides a description of a widget. \n\nAlso included is a package data with functions for GraphQL, relay and context from the golang.org/x/net/context package. \n\nThe "node" and "viewer" fields

In [39]:
question = "what is the input to the read"
result = qa.invoke({"input": question})
result["answer"]

'The input to the read is not explicitly mentioned in the given context. However, considering the context is related to GraphQL schemas and queries, typically the input to a read operation in GraphQL could be an ID or a set of parameters used to fetch specific data.'

In [40]:
question = "what is the output"
result = qa.invoke({"input": question})
result["answer"]

'The code provided is a collection of various JavaScript and React code snippets, so there\'s no specific output. However, here\'s a general idea of what some of the code does:\n\n1. The first block of code is a React component named `App` that renders a list of "widgets". It\'s using the Relay framework, a JavaScript framework for building data-driven React applications, to fetch the data.\n\n2. The second block of code is a part of a GraphQL schema (not fully shown), which defines the types and relationships between the types in your dataset.\n\n3. The third block of code is a JSON file which includes details about a project, including its name, description, repository, version, scripts, and dependencies.\n\n4. The fourth block of code is a JavaScript module that exports a function to generate a Relay Plugin using a provided schema.\n\n5. The fifth block of code is another part of a GraphQL schema.\n\n6. The sixth block of code is a Relay Route that defines a single route in the appl

In [41]:
question = "does the code handle any data transformation"
result = qa.invoke({"input": question})
result["answer"]

"The provided context does not indicate any data transformation. It mainly describes a GraphQL schema which is a structure that outlines the types of data that can be queried via a GraphQL API, but it doesn't perform any data transformation itself."

In [23]:
question = "provide me the data lineage"
result = qa.invoke({"input": question})
result["answer"]

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6


"The data in this code flows through a few key steps:\n\n1. Initiation: The code defines two types of data - User and Widget. User has attributes id, name, and widgets. Widget has attributes id and name. It also creates mock data for a User, and three Widgets.\n\n2. Data Access: Functions are defined to get the User by id (GetUser), get the Viewer (GetViewer), and get a Widget by id (GetWidget). Additionally, there's a function to get all Widgets (GetWidgets), and a function to turn a slice of Widgets into an interface slice (WidgetsToInterfaceSlice).\n\n3. GraphQL Types: GraphQL types are created for User and Widget. These types define how these entities are represented and interacted with in the GraphQL API. The User type includes an id and widgets field, the latter of which resolves to a connection. The Widget type includes an id and a name field.\n\n4. Schema Creation: A GraphQL schema is created which specifies the types and fields that can be queried. It includes a node field and

In [24]:
question = "what are the queries run here"
result = qa.invoke({"input": question})
result["answer"]

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6


"The main query run in this code is the schema introspection query, which is executed within the `main()` function of the first main package. The `RequestString` field of the `graphql.Params` struct is set to `testutil.IntrospectionQuery`, which is a predefined query that requests the details of the entire GraphQL schema. The result of this query is then saved as a JSON file.\n\nIn the data package, there are also resolvers defined for some fields in the `User` and `Query` types. These are not queries themselves, but define how the server should resolve these fields when they are included in a query. For example, the `viewer` field of the `Query` type is resolved by calling the `GetViewer` function, which returns a `User` object. The `widgets` field of the `User` type is resolved by converting the result of the `GetWidgets` function to a relay connection.\n\nIn addition, the `node` field in the `Query` type uses the `NodeField` from the relay's `nodeDefinitions` which is typically used

In [25]:
question = "draw the data lineage graph"
result = qa.invoke({"input": question})
result["answer"]

Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6


'As an AI text-based model, I\'m unable to draw visual diagrams. However, I can describe the data lineage in a text-based manner for your understanding. The data flow starts from the mock data that contains a User and a list of Widgets. \n\n1. The `GetUser(id string)` function retrieves a User by its ID.\n2. The `GetViewer()` function retrieves the mock User data.\n3. The `GetWidget(id string)` function retrieves a particular Widget by its ID.\n4. The `GetWidgets()` function retrieves all the Widgets.\n\nThis data is then used in the GraphQL schema:\n\n1. The `User` GraphQL object uses the `relay.GlobalIDField` for the "id" field and a custom field "widgets" which uses the `GetWidgets()` function to retrieve a list of widgets.\n2. The `Widget` GraphQL object uses the `relay.GlobalIDField` for the "id" field and has a custom "name" field.\n3. The Query object has a "node" field that uses the `relay.NodeField` and a "viewer" field that uses the `GetViewer()` function to retrieve the User